# 見積もり・データは一設計関連

In [80]:
# %pip install pandas
# %pip install altair
# %pip install tablate

---

## 論物変換

TODO:

## Buffer Layout

TODO:

## DMA Scheme

TODO:

---

## 誤り訂正

悩んだので ECC 内臓の上位モデルを参考に設計する

- w/o ECC (こちらを使用している) [KIOXIA_TC58NVG0S3HTA00](https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/506/KIOXIA_TC58NVG0S3HTA00_Rev2.00_E191001C.pdf)
- w/ ECC [KIOXIA_TC58BVG0S3HTA00](https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/506/KIOXIA_TC58BVG0S3HTA00_REV2.00_E20191001C.pdf)

### Datasheet確認

#### ECC & Sector definition for ECC

- 2KByte pageは 512byte * 4area + 16byte * 4area で使用している
- main + spare の 512+16=528byteごとに、 9bitの検出と8bitの訂正能力を備える
- ECC parity code は 2112-2175 (64byte) に格納するので、ここにはアクセスできないという注意書き

main+spareごとECCのペアになっている点、ECC Status Readの戻り値がArea事に分かれていること、最後のECC parity code格納から以下のように推測できる（実際は不明）

| sector | main [byte] | spare [byte] | ECC parity [byte] |
| ------ | ----------- | ------------ | ----------------- |
| 1st    | 0-511       | 2048 - 2063  | 2112 - 2127       |
| 2nd    | 512 - 1023  | 2064 - 2079  | 2128 - 2143       |
| 3rd    | 1024 - 1535 | 2080 - 2095  | 2144 - 2159       |
| 4th    | 1536 - 2047 | 2096 - 2111  | 2160 - 2175       |

(17) Reliability Guidance によると 8bit ECC for each 512byte 必要 (これは w/o ECC modelにも記載あり) とある。
(en Wikipediaより) additional parityありの拡張Hamming(256,247)だと考えると、data部247bit ごとにECC Parityが9bit付与されることになる。

512+12byte分付与すると20byte分のECC parity code格納が必要なので若干足りていないように見える。何か見落としがあるだろうか。


#### DC Balance? (bit scramble?)

同じ (17) Reliability Guidance で見つけたが0 data paddingをやめろという記載もある。このUSBメモリにはAll Zeroのバイナリを置かないでください、とは言えないので対策が必要そう。
だんだんCDの不揮発化フォーマット見たくなりつつあるような気もするが、データをアナログな物理現象に変換して格納し、何らかのエラーや物理都合の対策があるという点では味同じなのかもしれない。

何等かLFSRなどとのxorがパッと思いつくが、LFSRの出力の反転値がデータだとAll Zeroになるのでこれでは不十分。伝送路符号に当たる8b10bなどがパッと浮かぶのでこのあたりだろう。
気になる点が処理順である。ECC -> 8b/10b だと、復号化するときに化けたものをもとに8b/10b decodeして最初のエラーよりも大きな誤りが伝搬してしまうように考える。
そのため、元データ -> 8b/10bしたものを書き込む元データとして捉え、これをECC Encodeするのが良さそうだ。

64b66b,128b130bなどは少ない情報を漁る限りテーブルによる変換ではなく、出力フィードバックのある回路で構成しやすくなっている（Programming言語的には微妙かも）。
また、信号パターンが固定化したときの不要輻射対策に前処理に疑似乱数とのscrambleがあるらしい...。今回は繰り返しパターンは影響しない（と思っているけど情報なし）と仮定して、エンコードとプリアンブルの付与だけとする。
＃ プリアンブルも今回の用途ならいらないような気がしなくもない

#### ECC + bit scramble 組み合わせの見積もり

生成したECC Parity codeの置き場所も気になる。先述の表だと2112-2175byteに束ねられているように推察できる。今回はこれを見習うことにする。
以下Codeで見積もりしたPage内訳

|    |   dc_balance_encode_size.data_bits |   dc_balance_encode_size.total_bits |   ecc_encode_size.data_bits |   ecc_encode_size.parity_bits |   code_rate |   surplus_bytes |   max_ecc_error_detection_count |
|---:|-----------------------------------:|------------------------------------:|----------------------------:|------------------------------:|------------:|----------------:|--------------------------------:|
| 48 |                                128 |                                 130 |                         247 |                             8 |        0.95 |              21 |                              16 |


若干見づらいので書き直したものが以下。128b/130b EncodeのパフォーマンスやCode Sizeに問題がなければ、64b/66b Encodeは棄却して良さそうに見える。
ECCの保護単位を256byte,512byte (w/ parity) どちらで構成するかは、少ない単位で構成できていたほうが体制が上がりそう。一方で空きbyte数の余裕観点で管理データがどうしても21byteで収められないなら512byteで構成するしかない。


| | ECC                 | DC Balance | 自由に使える空きbyte | pageあたりのECC保護分割数    | 採択時のpros |
|-|---------------------|-----------:|--------------------:|----------------------------:|-------------|
|1|拡張Hamming(256, 247)| 128b/130b  | 21                  | 8.5                         |特別な理由がなければ採用。推奨ECC厚生よりも更に細分化して構成できているので、エラー多発時の訂正能力が高い|

#### CRC追加

どう使うかはあまり考えていなかったが（実装中に欲しくなったときに使う事になりそう）、先の採択prosについて考えていてECCの保護単位を細かくすることのデメリットを考えていなかった。
ずばり検査数が倍になるのである。ECC分割数分だけ検査が必要なのでParity計算と一致を確認しなければならない。

RP2040は小規模であまり余裕のないプロセッサだと思うので、xorで実装できるといえど2k dataの読み出しのたびにこの計算をやるのはちょっと辛い気がする。
そもそも訂正が必要かどうかに気づくことが高速に行えればよいので、CRCなどを付与するのは良いかと考えた。
幸いRP2040は (おそらくUSB IPで使用する目的だと思うが) DMAのSnoopでCRC計算ができるので、これを付与するのが良さそうに思う。

これまたEncode同様適用順序に悩むが、CRCはあくまで訂正の要否判断なので、最終的なデータでCRC検査できることが望ましい。
またDMAでSnoopする都合上、Data bufferの一番最初か最後に付与しておいて、それ以外全部の領域をチェインなし一度のDMA Kickで済ませられるとよさそう。
DMAのDataUnitSizeを4byteにすることも鑑みると、4byte単位で開けておく。


#### データ配置総括

所々で順序関係について書いてはいたが、全部まとめると以下の表になる。
表示機128b130bのプリアンブルなくても良い気がするので、もしかしたらあとから外してreserve扱いとするかもしれない。
＃ （Streamでの伝送ではないので...と思ったがData outputの構図はStreamっぽいきもしなくもない)


| description             | area [byte] | ecc: 拡張Hamming(256, 247) |
| ----------------------  | ----------- | -------------------------- |
| usb host data0          |    0 -  511 | 512byte data |
| dc balance extra0       |  512 -  519 | 128b/130b encode additional `512[byte]*(130[bit]/128[bit])-512[byte]=8[byte]`|
| usb host data1          |  520 - 1031 | 512byte data |
| dc balance extra1       | 1032 - 1039 | 128b/130b encode additional `512[byte]*(130[bit]/128[bit])-512[byte]=8[byte]`|
| usb host data2          | 1040 - 1551 | 512byte data |
| dc balance extra2       | 1552 - 1559 | 128b/130b encode additional `512[byte]*(130[bit]/128[bit])-512[byte]=8[byte]`|
| usb host data3          | 1560 - 2071 | 512byte data |
| dc balance extra2       | 2072 - 2079 | 128b/130b encode additional `512[byte]*(130[bit]/128[bit])-512[byte]=8[byte]`|
| reserved (for last ecc) | 2080 - 2091 | ECC保護単位の最後端数分。`align_u32(2080[byte]*8[bit/byte]/247[data bit/ecc unit]/8[bit/byte])=12[byte]` 分のあまり。|
| ecc parity code         | 2092 - 2171 | `2092[byte]*8[bit/byte]/247[data bit/ecc unit]=68[ecc unit], align_u32(68[ecc unit]*9[bit/ecc unit]/8[bit/byte])=80[byte]` |
| crc code                | 2172 - 2175 | 4byte crc (codeに4byte使わないなら0埋め) |

Data, DC Balance, ECCに属さないデータ部分は `reserved (for last ecc) 12byte` +  `CRC code 4byte` = 16byte。設計通り21byteいない


### 共通実装

In [14]:
import math
from dataclasses import asdict, dataclass, field

import pandas as pd

def data_align(data: int, align_bytes: int) -> int:
    """Align data to the next multiple of align_bytes."""
    return math.ceil(data/align_bytes)*align_bytes


@dataclass(frozen=True)
class DcBalanceEncodeSize:
    """
    DC Balance Encodeの設定
    """

    data_bits: int
    total_bits: int

    @property
    def code_rate(self) -> float:
        return self.data_bits / self.total_bits


@dataclass(frozen=True)
class EccEncodeSize:
    """
    ECCの設定
    """

    parity_bits: int
    additional_bits: int
    data_bits: int

    @property
    def total_bits(self) -> int:
        return self.data_bits + self.parity_bits + self.additional_bits

    @property
    def code_rate(self) -> float:
        return self.data_bits / self.total_bits

    @classmethod
    def from_parity_bits(
        cls, parity_bits: int, additional_bits: int = 1
    ) -> "EccEncodeSize":
        """
        Create an ECC frame size from the number of parity bits and additional bits.
        :param parity_bits: The number of parity bits.
        :param additional_bits: The number of additional bits.
        :return: The ECC frame size.
        """
        data_bits = 2**parity_bits - parity_bits - 1
        return cls(parity_bits, additional_bits, data_bits)


@dataclass(frozen=False)
class EncodeConfig:
    """
    NAND ICのPage不揮発化におけるエンコード設定
    """

    ###############################################################
    # ユーザー設定

    # DC Balance Encodeの設定
    dc_balance_encode_size: DcBalanceEncodeSize

    # ECCの設定
    ecc_encode_size: EccEncodeSize

    ###############################################################
    # ユーザー設定可能だが、基本変更する必要はない

    # USB Hostからのデータ最小単位
    logical_data_bytes: int = 512

    # 1ページあたりにUSB Hostからのデータを何個格納するか
    logical_data_num_per_page: int = 4

    # 1pageあたりのデータ量
    bytes_per_page = 2048 + 128

    # ECC保護すべき最小Parity size
    require_ecc_protection_bits: int = 8

    ###############################################################
    # __post_init__で計算する値

    # 1pageあたりのUSB Hostから書き込まれるデータサイズ
    logical_data_bytes_per_page: int = field(default=None)

    # 1pageあたりの冗長データ量
    redundant_bytes: int = field(default=None)

    # 符号化率
    code_rate: float = field(default=None)

    # logical_data_bytes_per_pageに対して、冗長データ量を割り当てたときの、合計のデータ量
    require_bytes: int = field(default=None)

    # 冗長データ量が足りるかどうか
    is_sufficient_total_bytes: int = field(default=None)

    # 冗長データで余るバイト数
    surplus_bytes: int = field(default=None)

    # 冗長データが足りないバイト数
    short_bytes: int = field(default=None)

    # ECC保護が十分かどうか
    is_sufficient_ecc_protection: int = field(default=None)

    # ECC保護で require_ecc_protection_bytes で指定された中で検出できるエラーの最大数. 保護範囲が複数に分かれる場合はそれぞれで発生する理想ケースを想定
    max_ecc_error_detection_count: int = field(default=None)

    # DC Balance Encodeの追加データ量
    data_encode_additional_bytes: int = field(default=None)

    # ECC保護対象の総数
    ecc_protection_entry_n: int = field(default=None)
    
    # ECC保護のための追加データ量
    ecc_protection_total_parity_bytes: int = field(default=None)

    # ECC保護したときの最後の端数のバイト数
    ecc_padding_bytes: int = field(default=None)

    def __post_init__(self):
        self.logical_data_bytes_per_page = (
            self.logical_data_bytes * self.logical_data_num_per_page
        )
        self.redundant_bytes = self.bytes_per_page - self.logical_data_bytes_per_page
        self.code_rate = (
            self.ecc_encode_size.code_rate * self.dc_balance_encode_size.code_rate
        )
        self.require_bytes = math.floor(
            self.logical_data_bytes_per_page * (1.0 / self.code_rate)
        )
        self.is_sufficient_total_bytes = self.require_bytes <= self.bytes_per_page
        self.surplus_bytes = (
            self.bytes_per_page - self.require_bytes
            if self.is_sufficient_total_bytes
            else 0
        )
        self.short_bytes = (
            self.require_bytes - self.bytes_per_page
            if not self.is_sufficient_total_bytes
            else 0
        )
        self.is_sufficient_ecc_protection = (
            self.ecc_encode_size.parity_bits == self.require_ecc_protection_bits
        )
        self.max_ecc_error_detection_count = math.floor(
            self.logical_data_bytes * 8 / self.ecc_encode_size.data_bits
        )

        # data size見積もり
        ecc_protection_bytes = self.logical_data_bytes * self.logical_data_num_per_page
        ecc_protection_bits = ecc_protection_bytes * 8

        self.data_encode_additional_bytes = math.ceil(self.logical_data_bytes * (self.dc_balance_encode_size.total_bits / self.dc_balance_encode_size.data_bits))-self.logical_data_bytes

        ecc_data_bits = self.ecc_encode_size.data_bits
        ecc_parity_bits = self.ecc_encode_size.parity_bits + c.ecc_encode_size.additional_bits

        self.ecc_protection_entry_n = math.ceil(ecc_protection_bits / ecc_data_bits)
        ecc_protection_total_parity_bits = self.ecc_protection_entry_n * ecc_parity_bits
        self.ecc_protection_total_parity_bytes = data_align(ecc_protection_total_parity_bits / 8, align_bytes=4) 

        ecc_padding_bytes = ecc_protection_bits % ecc_data_bits
        self.ecc_padding_bytes = data_align(ecc_padding_bytes / 8, align_bytes=4)


### Data Layout設計時のreserved見積もり

In [12]:

calc_config_list = [
    EncodeConfig(dc_balance_encode_size=DcBalanceEncodeSize(128, 130), ecc_encode_size=EccEncodeSize.from_parity_bits(8)),
    EncodeConfig(dc_balance_encode_size=DcBalanceEncodeSize(128, 130), ecc_encode_size=EccEncodeSize.from_parity_bits(9)),
    EncodeConfig(dc_balance_encode_size=DcBalanceEncodeSize(64, 66), ecc_encode_size=EccEncodeSize.from_parity_bits(9)),
]

for c in calc_config_list:
    print(c)
    print(f"data_encode_additional_bytes={c.data_encode_additional_bytes}")
    print(f"ecc_protection_entry_n={c.ecc_protection_entry_n}")
    print(f"ecc_protection_total_parity_bytes={c.ecc_protection_total_parity_bytes}")
    print(f"ecc_padding_bytes={c.ecc_padding_bytes}")
    print("")

EncodeConfig(dc_balance_encode_size=DcBalanceEncodeSize(data_bits=128, total_bits=130), ecc_encode_size=EccEncodeSize(parity_bits=8, additional_bits=1, data_bits=247), logical_data_bytes=512, logical_data_num_per_page=4, require_ecc_protection_bits=8, logical_data_bytes_per_page=2048, redundant_bytes=128, code_rate=0.9500000000000001, require_bytes=2155, is_sufficient_total_bytes=True, surplus_bytes=21, short_bytes=0, is_sufficient_ecc_protection=True, max_ecc_error_detection_count=16, data_encode_additional_bytes=8, ecc_protection_entry_n=67, ecc_protection_total_parity_bytes=76, ecc_padding_bytes=12)
data_encode_additional_bytes=8
ecc_protection_entry_n=67
ecc_protection_total_parity_bytes=76
ecc_padding_bytes=12

EncodeConfig(dc_balance_encode_size=DcBalanceEncodeSize(data_bits=128, total_bits=130), ecc_encode_size=EccEncodeSize(parity_bits=9, additional_bits=1, data_bits=502), logical_data_bytes=512, logical_data_num_per_page=4, require_ecc_protection_bits=8, logical_data_bytes_per

### DC Balance Encode + ECC Encode時の効率・余剰byte数計算

In [16]:
import altair as alt
from tabulate import tabulate

pd.set_option("display.max_columns", 256)
pd.set_option("display.max_rows", 256)

# 見積もり
dc_balance_encode_size_list = [
    DcBalanceEncodeSize(8, 10),
    DcBalanceEncodeSize(64, 66),
    DcBalanceEncodeSize(64, 67),
    DcBalanceEncodeSize(128, 130),
    # DcBalanceEncodeSize(1, 1),  # disable
]

parity_len_n = 12  # 計算対象にするparity長
ecc_encode_size_list = [
    EccEncodeSize.from_parity_bits(i) for i in range(2, parity_len_n)
]

encode_config_list = [
    EncodeConfig(dc_balance_encode_size=dc, ecc_encode_size=ecc)
    for dc in dc_balance_encode_size_list
    for ecc in ecc_encode_size_list
]
encode_config_df = pd.json_normalize(([asdict(c) for c in encode_config_list]))


def print_chart(df: pd.DataFrame):
    base_chart_1 = (
        alt.Chart(df)
        .mark_point()
        .encode(
            alt.X("ecc_encode_size\\.data_bits:N"),
            alt.Row("dc_balance_encode_size\\.data_bits:N"),
        )
        .properties(width=300, height=100)
    )
    base_chart_2 = (
        alt.Chart(df)
        .mark_rect()
        .encode(
            alt.X("ecc_encode_size\\.data_bits:N"),
            alt.Y("dc_balance_encode_size\\.data_bits:N"),
        )
        .properties(width=300, height=300)
    )
    base_chart_3 = (
        alt.Chart(df)
        .mark_point()
        .encode(
            alt.X("ecc_encode_size\\.data_bits:N"),
            alt.Row("dc_balance_encode_size\\.data_bits:N"),
        )
        .properties(width=300, height=100)
    )
    alt.vconcat(
        # 横軸: ECCのデータビット数, 縦軸: 冗長データ量, 行: DC Balance Encodeのデータビット数
        alt.hconcat(
            base_chart_1.encode(
                alt.Y("surplus_bytes:Q"),
                alt.Color("is_sufficient_total_bytes:N"),
            ),
            base_chart_1.encode(
                alt.Y("code_rate:Q"),
                alt.Color("is_sufficient_ecc_protection:N"),
            ),
        ).resolve_scale(color="independent"),
        # Encode設定と余裕度のヒートマップ
        alt.hconcat(
            base_chart_2.encode(
                alt.Color("surplus_bytes:Q"),
            ),
            base_chart_2.encode(
                alt.Color("code_rate:Q"),
            ),
        ).resolve_scale(color="independent"),
        # 余剰byte数、Padding余りbyte数
        alt.hconcat(
            base_chart_3.encode(
                alt.Y("ecc_padding_bytes:Q"),
                alt.Color("is_sufficient_total_bytes:N"),
            ),
            base_chart_3.encode(
                alt.Y("ecc_protection_entry_n:Q"),
                alt.Color("is_sufficient_ecc_protection:N"),
            ),
        ).resolve_scale(color="independent"),
    ).resolve_scale(color="independent").display()


# 今回の要件を満たせるもの
available_encode_config_df = encode_config_df.query(
    "is_sufficient_total_bytes and is_sufficient_ecc_protection"
)
print(
    tabulate(
        available_encode_config_df[
            [
                "dc_balance_encode_size.data_bits",
                "dc_balance_encode_size.total_bits",
                "ecc_encode_size.data_bits",
                "ecc_encode_size.parity_bits",
                # "ecc_encode_size.additional_bits",
                "code_rate",
                "surplus_bytes",
                "max_ecc_error_detection_count",
            ]
        ],
        headers="keys",
        tablefmt="pipe",
    )
)

# 全体
print_chart(encode_config_df)
print_chart(available_encode_config_df)
available_encode_config_df


|    |   dc_balance_encode_size.data_bits |   dc_balance_encode_size.total_bits |   ecc_encode_size.data_bits |   ecc_encode_size.parity_bits |   code_rate |   surplus_bytes |   max_ecc_error_detection_count |
|---:|-----------------------------------:|------------------------------------:|----------------------------:|------------------------------:|------------:|----------------:|--------------------------------:|
| 36 |                                128 |                                 130 |                         247 |                             8 |        0.95 |              21 |                              16 |


alt.VConcatChart(...)

alt.VConcatChart(...)

,logical_data_bytes,logical_data_num_per_page,require_ecc_protection_bits,logical_data_bytes_per_page,redundant_bytes,code_rate,require_bytes,is_sufficient_total_bytes,surplus_bytes,short_bytes,is_sufficient_ecc_protection,max_ecc_error_detection_count,data_encode_additional_bytes,ecc_protection_entry_n,ecc_protection_total_parity_bytes,ecc_padding_bytes,dc_balance_encode_size.data_bits,dc_balance_encode_size.total_bits,ecc_encode_size.parity_bits,ecc_encode_size.additional_bits,ecc_encode_size.data_bits
36,512,4,8,2048,128,0.95,2155,True,21,0,True,16,8,67,76,12,128,130,8,1,247
